In [10]:
import json
import requests
import pandas as pd
from riotwatcher import LolWatcher, ApiError

with open('data/api-key.txt', 'r') as api:
    API_KEY = api.read()
lol_watcher = LolWatcher(API_KEY)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}

keyid = pd.read_csv("data/champion-key-ids.csv")

In [48]:
def get_matches(region, summ_name, champion_key="", index=""):
    summoner_obj = lol_watcher.summoner.by_name(region, summ_name)
    if champion_key != "":
        champion_key = "".join(["?champion=",str(champion_key)])
    if index!="":
        index = "".join(["&beginIndex=", str(index)])
    if region == "kr":
        summ_url = "".join(["https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/",
                           summoner_obj["accountId"], champion_key, index])
    elif region == "na":
        summ_url = "".join(["https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/",
                           summoner_obj["accountId"], champion_key, index])
    summ_response = requests.get(summ_url,headers=HEADERS)
    summ_json = json.loads(summ_response.text)
    try:
        return pd.DataFrame(summ_json["matches"])
    except KeyError as e:
        print(summ_response)
        raise e

In [49]:
get_matches("kr", "Gen G Clid", 64)

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4748107995,64,420,13,1603960828768,NONE,JUNGLE
1,KR,4746315492,64,420,13,1603871800200,DUO_SUPPORT,NONE
2,KR,4744152493,64,420,13,1603781671413,NONE,JUNGLE
3,KR,4742500253,64,420,13,1603703713832,NONE,JUNGLE
4,KR,4742168190,64,420,13,1603693491352,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
95,KR,4448628377,64,420,13,1592147683134,NONE,JUNGLE
96,KR,4446420179,64,420,13,1592069157211,NONE,JUNGLE
97,KR,4446373912,64,420,13,1592066953874,NONE,JUNGLE
98,KR,4446069318,64,420,13,1592059655257,NONE,JUNGLE


In [50]:
get_matches("kr", "Gen G Clid", 64, 100)

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4436066469,64,420,13,1591632756693,NONE,JUNGLE
1,KR,4435915493,64,420,13,1591628228703,DUO_SUPPORT,NONE
2,KR,4435741399,64,420,13,1591623959574,DUO_SUPPORT,NONE
3,KR,4434408444,64,420,13,1591552381007,NONE,JUNGLE
4,KR,4431672817,64,420,13,1591452818127,DUO_SUPPORT,NONE
...,...,...,...,...,...,...,...,...
95,KR,4111590787,64,420,13,1580043000935,NONE,JUNGLE
96,KR,4110995062,64,420,13,1580028540222,NONE,JUNGLE
97,KR,4103855000,64,420,13,1579753453932,NONE,JUNGLE
98,KR,4103185314,64,420,13,1579713633765,NONE,JUNGLE


In [57]:
def get_all_matches(region, summ_name, champion_key):
    index = 0
    match_batches = []
    while True:
        current = get_matches(region, summ_name, champion_key, index)
        if len(current)==0:
            break
        match_batches.append(current)
        index+=100
    return pd.concat(match_batches)

In [58]:
matches = get_all_matches("kr", "Gen G Clid", 64)

In [59]:
matches

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4748107995,64,420,13,1603960828768,NONE,JUNGLE
1,KR,4746315492,64,420,13,1603871800200,DUO_SUPPORT,NONE
2,KR,4744152493,64,420,13,1603781671413,NONE,JUNGLE
3,KR,4742500253,64,420,13,1603703713832,NONE,JUNGLE
4,KR,4742168190,64,420,13,1603693491352,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
51,KR,3407937944,64,420,11,1541265799137,NONE,JUNGLE
52,KR,3405384450,64,420,11,1541149020834,NONE,JUNGLE
53,KR,3404759665,64,420,11,1541076588408,NONE,JUNGLE
54,KR,3404761384,64,420,11,1541074919315,NONE,JUNGLE


In [60]:
matches = matches[matches["lane"]=="JUNGLE"]

In [61]:
matches.to_csv("data/clid-matches.csv", index=False)